In [1]:
import lsst.daf.butler as daf_butler
import numpy as np
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import astropy.units as u
afwDisplay.setDefaultBackend("firefly")

In [2]:
collections=["u/elhoward/TOO/0725J","u/rea3/test_0725J_v10/20250729T163637Z","u/elhoward/TOO/0725J/MMADS_candidates","LSSTCam/runs/nightlyValidation/20250728/w_2025_30/DM-50157","u/elhoward/TOO/0725J/DESGW_candidates_v2","u/elhoward/TOO/0725J/test"]

repo="embargo_new"

In [3]:
MyBut = daf_butler.Butler(repo,collections=collections)

In [4]:
finalVisits = []
for k in np.arange(2025072800099,2025072800159):
    if int(str(k)[-1])!=9:
        finalVisits.append(k)

In [5]:
detector = 42
visit = 2025072800398

In [6]:
# MyBut.registry.queryCollections("*ightly*20250728*")

In [7]:
diffRefs = list(MyBut.query_datasets("difference_image",where=f"exposure={visit} and instrument='LSSTCam' and detector={detector}"))
tempRefs = list(MyBut.query_datasets("template_matched",where=f"exposure={visit} and instrument='LSSTCam' and detector={detector}"))
searchRefs = list(MyBut.query_datasets("preliminary_visit_image",where=f"exposure={visit} and instrument='LSSTCam' and detector={detector}"))

In [8]:
diff = MyBut.get(diffRefs[0])
temp = MyBut.get(tempRefs[0])
search = MyBut.get(searchRefs[0])

In [21]:
display = afwDisplay.Display(backend="firefly",frame=0)
display.image(search)

In [10]:
display = afwDisplay.Display(backend="firefly",frame=1)
display.image(temp)

In [16]:
display = afwDisplay.Display(backend="firefly",frame=2)
display.image(diff)